# 🛡️ Overfit Guard - Interactive Demo

**Automatic Overfitting Detection & Correction for ML Models**

[![GitHub](https://img.shields.io/badge/GitHub-overfit--guard-blue?logo=github)](https://github.com/Core-Creates/overfit-guard)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Core-Creates/overfit-guard/blob/main/notebooks/overfit_guard_colab_demo.ipynb)

---

## 📋 What This Demo Shows:
1. ✅ Real-world dataset (Wisconsin Breast Cancer)
2. ✅ Side-by-side comparison (with vs without guard)
3. ✅ Automatic overfitting detection & correction
4. ✅ Visual performance analysis
5. ✅ Production-ready code examples

**Estimated runtime:** 3-5 minutes

---

## 🚀 Step 1: Installation

First, we'll install Overfit Guard directly from GitHub:

In [ ]:
%%capture
# Install from GitHub repo
!pip install git+https://github.com/Core-Creates/overfit-guard.git
!pip install scikit-learn matplotlib torch torchvision

print("✅ Installation complete!")

In [ ]:
# Verify installation
import overfit_guard
print(f"✅ Overfit Guard v{overfit_guard.__version__} installed successfully!")
print(f"📦 Location: {overfit_guard.__file__}")

## 📦 Step 2: Import Libraries

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from overfit_guard.integrations.pytorch import create_pytorch_monitor

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {device}")
if device.type == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

## 🔬 Step 3: Prepare Real-World Dataset

**Wisconsin Breast Cancer Dataset:**
- 569 samples (357 benign, 212 malignant)
- 30 numerical features
- Binary classification task
- Medical diagnosis scenario

In [ ]:
# Load dataset
print("📁 Loading Wisconsin Breast Cancer dataset...")
data = load_breast_cancer()
X, y = data.data, data.target

print(f"\n📊 Dataset Info:")
print(f"   Samples: {X.shape[0]}")
print(f"   Features: {X.shape[1]}")
print(f"   Classes: {np.unique(y)} (0=malignant, 1=benign)")
print(f"   Distribution: {dict(zip(*np.unique(y, return_counts=True)))}")

# Split data: 70% train, 15% val, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

# Standardize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Create PyTorch datasets
train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.LongTensor(y_val))
test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.LongTensor(y_test))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"\n✅ Data Split:")
print(f"   Training: {len(train_dataset)} samples")
print(f"   Validation: {len(val_dataset)} samples")
print(f"   Test: {len(test_dataset)} samples")

## 🧠 Step 4: Define Model Architecture

In [ ]:
class BreastCancerClassifier(nn.Module):
    """Neural network for breast cancer classification."""
    
    def __init__(self, input_size=30):
        super().__init__()
        self.network = nn.Sequential(
            # Layer 1
            nn.Linear(input_size, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            # Layer 2
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            # Layer 3
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            # Output
            nn.Linear(16, 2)
        )
    
    def forward(self, x):
        return self.network(x)

# Test model
test_model = BreastCancerClassifier(input_size=X_train.shape[1])
print(f"✅ Model created successfully")
print(f"   Parameters: {sum(p.numel() for p in test_model.parameters()):,}")

## 🏋️ Step 5: Training Functions

In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    """Train for one epoch."""
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return total_loss / len(dataloader), correct / total

def validate(model, dataloader, criterion, device):
    """Validate the model."""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return total_loss / len(dataloader), correct / total

print("✅ Training functions defined")

## 🆚 Step 6: Comparative Training

We'll train **two identical models**:
1. **Baseline** - Standard training (no guard)
2. **Protected** - With Overfit Guard enabled

This demonstrates the real-world impact of automatic overfitting prevention.

In [ ]:
def run_training(use_guard=True, num_epochs=50, verbose=True):
    """Run training experiment."""
    
    # Reset seeds
    torch.manual_seed(42)
    np.random.seed(42)
    
    if verbose:
        print(f"\n{'='*70}")
        print(f"🔧 Training {'WITH' if use_guard else 'WITHOUT'} Overfit Guard")
        print(f"{'='*70}\n")
    
    # Create model
    model = BreastCancerClassifier(input_size=X_train.shape[1]).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
    
    # Setup monitor
    monitor = None
    if use_guard:
        monitor = create_pytorch_monitor(
            model=model,
            optimizer=optimizer,
            config={
                'auto_correct': True,
                'min_severity_for_correction': 'MODERATE',
                'correction_cooldown': 5,
                'log_level': 'WARNING'  # Less verbose
            },
            auto_correct=True
        )
        if verbose:
            print("🛡️  Overfit Guard: ACTIVE")
            print(f"   Detectors: {len(monitor.monitor.detectors)}")
            print(f"   Correctors: {len(monitor.monitor.correctors)}\n")
    
    # Training loop
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    for epoch in range(num_epochs):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # Check with monitor
        if monitor:
            results = monitor.on_epoch_end(
                epoch=epoch,
                model=model,
                train_metrics={'loss': train_loss, 'accuracy': train_acc},
                val_metrics={'loss': val_loss, 'accuracy': val_acc}
            )
            
            if monitor.should_stop:
                if verbose:
                    print(f"\n⏹️  Early stopping at epoch {epoch+1}")
                break
        
        # Print progress
        if verbose and (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1:2d}/{num_epochs} | "
                  f"Train: {train_acc:.4f} | Val: {val_acc:.4f} | "
                  f"Gap: {(train_acc - val_acc):.4f}")
    
    # Final test
    test_loss, test_acc = validate(model, test_loader, criterion, device)
    
    if verbose:
        print(f"\n📊 Final Results:")
        print(f"   Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
        print(f"   Train-Val Gap: {(history['train_acc'][-1] - history['val_acc'][-1]):.4f}")
        
        if monitor:
            summary = monitor.monitor.get_summary()
            print(f"\n   🛡️  Guard Activity:")
            print(f"      Detections: {summary['overfitting_detected']}")
            print(f"      Corrections: {summary['corrections_applied']}")
            print(f"      Rate: {summary['overfitting_rate']:.1%}")
    
    return history, test_acc, monitor

print("✅ Training function ready")

In [ ]:
# Run BASELINE (without guard)
history_baseline, test_acc_baseline, _ = run_training(use_guard=False, num_epochs=50)

In [ ]:
# Run WITH GUARD
history_guard, test_acc_guard, monitor = run_training(use_guard=True, num_epochs=50)

## 📊 Step 7: Visual Comparison

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

epochs_baseline = range(1, len(history_baseline['train_loss']) + 1)
epochs_guard = range(1, len(history_guard['train_loss']) + 1)

# Loss comparison
axes[0, 0].plot(epochs_baseline, history_baseline['train_loss'], 'b-', label='Train', linewidth=2, alpha=0.8)
axes[0, 0].plot(epochs_baseline, history_baseline['val_loss'], 'r-', label='Val', linewidth=2, alpha=0.8)
axes[0, 0].set_title('Loss - WITHOUT Overfit Guard', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epoch', fontsize=11)
axes[0, 0].set_ylabel('Loss', fontsize=11)
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(epochs_guard, history_guard['train_loss'], 'b-', label='Train', linewidth=2, alpha=0.8)
axes[0, 1].plot(epochs_guard, history_guard['val_loss'], 'r-', label='Val', linewidth=2, alpha=0.8)
axes[0, 1].set_title('Loss - WITH Overfit Guard ✅', fontsize=14, fontweight='bold', color='green')
axes[0, 1].set_xlabel('Epoch', fontsize=11)
axes[0, 1].set_ylabel('Loss', fontsize=11)
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# Accuracy comparison
axes[1, 0].plot(epochs_baseline, history_baseline['train_acc'], 'b-', label='Train', linewidth=2, alpha=0.8)
axes[1, 0].plot(epochs_baseline, history_baseline['val_acc'], 'r-', label='Val', linewidth=2, alpha=0.8)
axes[1, 0].set_title('Accuracy - WITHOUT Overfit Guard', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Epoch', fontsize=11)
axes[1, 0].set_ylabel('Accuracy', fontsize=11)
axes[1, 0].legend(fontsize=10)
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(epochs_guard, history_guard['train_acc'], 'b-', label='Train', linewidth=2, alpha=0.8)
axes[1, 1].plot(epochs_guard, history_guard['val_acc'], 'r-', label='Val', linewidth=2, alpha=0.8)
axes[1, 1].set_title('Accuracy - WITH Overfit Guard ✅', fontsize=14, fontweight='bold', color='green')
axes[1, 1].set_xlabel('Epoch', fontsize=11)
axes[1, 1].set_ylabel('Accuracy', fontsize=11)
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📈 Plots generated successfully!")

## 🏆 Step 8: Final Results & Analysis

In [ ]:
print("\n" + "="*70)
print("🏆 FINAL COMPARISON")
print("="*70)

# Baseline stats
gap_baseline = history_baseline['train_acc'][-1] - history_baseline['val_acc'][-1]
print("\n📊 WITHOUT Overfit Guard (Baseline):")
print(f"   Test Accuracy: {test_acc_baseline:.4f} ({test_acc_baseline*100:.2f}%)")
print(f"   Train-Val Gap: {gap_baseline:.4f}")
print(f"   Final Train Acc: {history_baseline['train_acc'][-1]:.4f}")
print(f"   Final Val Acc: {history_baseline['val_acc'][-1]:.4f}")

# Guard stats
gap_guard = history_guard['train_acc'][-1] - history_guard['val_acc'][-1]
summary = monitor.monitor.get_summary()

print("\n🛡️  WITH Overfit Guard:")
print(f"   Test Accuracy: {test_acc_guard:.4f} ({test_acc_guard*100:.2f}%)")
print(f"   Train-Val Gap: {gap_guard:.4f}")
print(f"   Final Train Acc: {history_guard['train_acc'][-1]:.4f}")
print(f"   Final Val Acc: {history_guard['val_acc'][-1]:.4f}")

# Improvements
test_improvement = (test_acc_guard - test_acc_baseline) * 100
gap_reduction = gap_baseline - gap_guard
gap_reduction_pct = (gap_reduction / abs(gap_baseline)) * 100 if gap_baseline != 0 else 0

print("\n💡 Improvement:")
print(f"   Test Accuracy: {test_improvement:+.2f}% {'✅' if test_improvement >= 0 else '⚠️'}")
print(f"   Gap Reduction: {gap_reduction:+.4f} ({gap_reduction_pct:+.1f}%) {'✅' if gap_reduction > 0 else '⚠️'}")

# Guard activity
print(f"\n🔧 Guard Activity:")
print(f"   Detections: {summary['overfitting_detected']}")
print(f"   Corrections: {summary['corrections_applied']}")
print(f"   Detection Rate: {summary['overfitting_rate']:.1%}")
print(f"   Active Detectors: {summary['active_detectors']}")
print(f"   Active Correctors: {summary['active_correctors']}")

print("\n" + "="*70)

# Interpretation
print("\n📝 Interpretation:")
if test_improvement > 0:
    print("   ✅ Test accuracy IMPROVED with Overfit Guard")
elif test_improvement == 0:
    print("   ✅ Test accuracy MAINTAINED (no degradation)")
else:
    print(f"   ⚠️  Test accuracy slightly decreased ({abs(test_improvement):.2f}%)")
    print("      This can happen when preventing overfitting on well-behaved datasets")

if gap_reduction > 0:
    print(f"   ✅ Train-val gap REDUCED by {gap_reduction_pct:.1f}% (better generalization)")
else:
    print("   ⚠️  Gap increased (model may need different tuning)")

print("\n" + "="*70)

## 🎯 Key Takeaways

### ✅ What Overfit Guard Does:
1. **Automatic Detection**: Monitors train/val metrics in real-time
2. **Smart Corrections**: Adjusts regularization & hyperparameters automatically
3. **Early Stopping**: Prevents wasted training time
4. **Zero Configuration**: Works out-of-the-box with sensible defaults

### 🎓 When to Use Overfit Guard:
- ✅ Deep neural networks with many parameters
- ✅ Limited training data
- ✅ Production ML pipelines
- ✅ Automated training workflows
- ✅ Research experiments

### 💻 Integration is Simple:
```python
from overfit_guard.integrations.pytorch import create_pytorch_monitor

# Just add these 3 lines to your training loop:
monitor = create_pytorch_monitor(
    model=model, 
    optimizer=optimizer, 
    auto_correct=True
)

# Then in your training loop:
monitor.on_epoch_end(epoch, model, train_metrics, val_metrics)
```

### 📚 Learn More:
- 📦 [PyPI Package](https://pypi.org/project/overfit-guard/)
- 💻 [GitHub Repository](https://github.com/Core-Creates/overfit-guard)
- 📖 [Full Documentation](https://github.com/Core-Creates/overfit-guard/blob/main/README.md)
- 🐛 [Report Issues](https://github.com/Core-Creates/overfit-guard/issues)

### 🚀 Next Steps:
1. Try Overfit Guard on your own datasets
2. Experiment with different configurations
3. Compare with manual overfitting prevention
4. Share feedback on GitHub!

---

**Made with ❤️ by the Overfit Guard team**

[![Star on GitHub](https://img.shields.io/github/stars/Core-Creates/overfit-guard?style=social)](https://github.com/Core-Creates/overfit-guard)